In [2]:
import google.datalab.bigquery as bq

In [16]:
compute_alpha="""
#standardSQL
SELECT
  SAFE_DIVIDE(SUM(arrival_delay*departure_delay), SUM(departure_delay*departure_delay)) as alpha
FROM
(
SELECT
  RAND() as my_splitter,
  arrival_delay,
  departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport='DEN' AND arrival_airport='LAX'
)

WHERE my_splitter < 0.8
"""

In [18]:
results = bq.Query(compute_alpha).execute().result().to_dataframe()
# print results
alpha=results['alpha'][0]
print alpha

0.9760437790121071


In [28]:
test="""
SELECT
  IF( RAND() < 0.8 , 'TRAIN', 'EVAL') as dataset,
  arrival_delay,
  departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE departure_airport='DEN' 
    AND arrival_airport='LAX'
"""
# bq.Query(test).execute().result()

dataset,arrival_delay,departure_delay
EVAL,1168.0,1170.0
TRAIN,1132.0,1144.0
EVAL,1102.0,1109.0
TRAIN,934.0,936.0
TRAIN,933.0,932.0
TRAIN,899.0,908.0
TRAIN,778.0,789.0
EVAL,792.0,788.0
EVAL,743.0,777.0
TRAIN,765.0,770.0


In [30]:
## Computing RMSE. Running this code block generates different outputs everytime (as a virtue of RAND()), try it
compute_rmse="""
SELECT
  dataset,
  SQRT(AVG((arrival_delay - ALPHA*departure_delay)*(arrival_delay - ALPHA*departure_delay))) as rmse
FROM
(
SELECT
  IF( RAND() < 0.8 , 'TRAIN', 'EVAL') as dataset,
  arrival_delay,
  departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE departure_airport='DEN' AND arrival_airport='LAX'
)
GROUP BY
  dataset
"""
bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result()

dataset,rmse
EVAL,13.0372652004
TRAIN,13.0953046713


Getting the same data set for evaluation and training : add both the queries together so that the datasets don't vary.
However, this is not going to choose the same pair of training and evaluation dataset the next time, so the results will change

In [33]:
train_and_eval_together="""
WITH
  alldata as (
  SELECT
    IF( RAND () < 0.8, 'train', 'eval') as dataset,
    arrival_delay,
    departure_delay
  FROM
    `bigquery-samples.airline_ontime_data.flights`
  WHERE
    arrival_airport='LAX' AND departure_airport='DEN' 
  ),
  training as (
  SELECT
  SAFE_DIVIDE(SUM(arrival_delay*departure_delay), SUM(departure_delay*departure_delay)) AS alpha
  FROM
    alldata
  WHERE
    dataset = 'train'
  )

SELECT
  MAX(alpha) as ALPHA,
  dataset,
  SQRT(AVG((arrival_delay - ALPHA*departure_delay)*(arrival_delay - ALPHA*departure_delay))) AS rmse
  FROM 
    alldata,
    training
  GROUP BY
    dataset
  """
bq.Query(train_and_eval_together).execute().result()

ALPHA,dataset,rmse
0.975254834947,train,13.0487117183
0.975254834947,eval,13.2215595038


In [35]:
compute_alpha="""
#standarSQL
SELECT
  SAFE_DIVIDE(SUM(arrival_delay*departure_delay), SUM(departure_delay*departure_delay)) as alpha
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport='DEN'
  AND arrival_airport='LAX'
  AND MOD(ABS(FARM_FINGERPRINT(date)), 10) < 8
"""
results = bq.Query(compute_alpha).execute().result().to_dataframe()
alpha = results['alpha'][0]
print alpha           ## this will remain constant, unless you don't change your algoritm

0.9758039143620403


In [41]:
## rmse. This will remain constant, until you don't change your algorithm
compute_rmse="""
SELECT
  IF(MOD(ABS(FARM_FINGERPRINT(date)),10)<8, 'train' , 'eval') AS dataset,
  SQRT(AVG((arrival_delay - ALPHA*departure_delay)*(arrival_delay - ALPHA*departure_delay))) AS rmse
FROM 
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport='DEN'
  AND arrival_airport='LAX'
GROUP BY
  dataset
"""
bq.Query(compute_rmse.replace('ALPHA', str(alpha))).execute().result()

dataset,rmse
train,13.1607120015
eval,12.7646852443
